# 0. selenium 설치

In [1]:
!pip install chromedriver_autoinstaller

In [2]:
!pip install selenium

# 1. 라이브러리 불러오기

In [3]:
# 라이브러리
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함


from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 크롤링

In [4]:
def open_chrome():
    ## 크롬 윈도우 설정
    # 크롬 드라이버 위치 설정
    chrome_path = chromedriver_autoinstaller.install()


    # 크롬 옵션
    options = webdriver.ChromeOptions()
    # 크롬 윈도우 사이즈 조절
    options.add_argument("--window-size=800,1400")

    driver = webdriver.Chrome(chrome_path, options=options)

    # 실행할 시간 랜덤값 지정
    A = np.random.randint(1,3)
    
    return driver , A

##### 참고


- find_elements_by_tag_name:          Copy Element의 tag
- find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
- find_elements_by_id:                Copy Element의 id attribute
- find_elements_by_class_name:        Copy Element의 class attribute
- find_elements_by_link_text:         Copy Element의 text
- find_elements_by_partial_link_text: Copy Element의 text중 일부
- find_elements_by_css_selector:      Copy selector


- '요소' 클릭:        	.click('')   
- '요소' 더블 클릭:	     .double_click('')   
- '텍스트' 입력:	         .send_keys('')  
- '요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
- '요소'로 마우스 이동:	  .move_to_element('')
- 스크롤 내리기:	         .execute_script("window.scrollto();")
- 실행:                  .perform()

## 2. 선수 데이터 크롤링 (시즌 & 클럽 경력 추가 part)


각 선수별 시즌, 클럽 경력을 가져온 뒤, 'name'과 세부 능력치 기준으로 merge 예정!

In [5]:
# 선수명 csv파일 불러오기
player_name = pd.read_csv('./data/player_info/player.csv')
search_name_list = list(player_name['play_name'])

In [6]:
# 중복값 제거
search_name_list = list(set(search_name_list))

> 정지된 기록

<br>

|구간|원인|비고|
|---|---|---|
|0 ~ 1233|        * 카톡 로그아웃||
|1234 ~ 14067|    * 검색 결과 오류  |    +동명이인 1차 업데이트: '동명이인' 출력|
|14068 ~ 20243|   * 검색 결과 오류 |     +동명이인 2차 업데이트: +'temp_name'|
|20244 ~ 22505|   * 검색 결과 오류|      +동명이인 3차 업데이트: +'search_name'|
|22506 ~ 23449|   * 검색 결과 오류|  'N. 메사투' 검색결과 없음 (number = 23450)|
|23451 ~ 25034|   * 검색 결과 오류||
|25035 ~ 30961|   * 사용할 수 없는 페이지|'검색 결과 오류' 예외처리 업데이트|
|30962 ~ 32411|   * 검색어 입력 에러||
|32412 ~ 36644|   * 완료!||

<br><br>

> 후처리 과정에서의 편의를 위해 다시 크롤링 진행 (~ 20243)

by 동명이인 3차 업데이트 ver.

<br>

|구간|원인|비고|
|---|---|---|
|0 ~ |||
| ~ |||
| ~ |||


In [115]:

# 정지된 위치 확인
def find_end(last_search_name):
    for i in range(len(search_name_list)):
        if search_name_list[i] == last_search_name:
            print(i)


find_end('Y. 바르나위')

32411


In [7]:
# 클럽 활동 연도 & 클럽 팀 합치기
def year_and_team_merge(temp_club_career):
    global temp_club_career_2
    temp_club_career_2 = []
    year = []
    team = []
    

    for i in range(1,len(temp_club_career.split('\n'))):
        if i % 2 == 0:
            year.append(temp_club_career.split('\n')[i])
        else:
            team.append(temp_club_career.split('\n')[i])


    for i in range(len(year)):
        temp_club_career_2.append(year[i]+' / '+team[i])
        
    return temp_club_career_2

In [117]:
# chrome 창 열기
driver, A = open_chrome()

## 선수 정보 >> 각 선수의 시즌 & 클럽 경력 크롤링
# 선수 정보 리스트
name = []
season = []
club_career = []
pay_side = []
overall = []
Sprint_speed = []
Acceleration =[]
Strength = []
Stamina = []
error_name = []

# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter'
driver.get(url)

# # 팝업창 닫기
# driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
# time.sleep(A)

# 정지된 위치부터 다시 시작 
for search_name in tqdm(search_name_list[:20243]):
   try:
      # 선수명 입력 & 검색
      driver.find_element_by_class_name('ui-autocomplete-input').click()
      driver.find_element_by_class_name('ui-autocomplete-input').send_keys(search_name)
      driver.find_element_by_class_name('btn_search').click()
      time.sleep(3)


      # 세부 설명 들어가기
      driver.find_element_by_class_name('name').click()
      driver.find_element_by_class_name('btn_detail_link').click()
      time.sleep(1)
      
      
      # 창 변경
      driver.switch_to.window(driver.window_handles[-1])
      time.sleep(1)
      
      
      # 클럽 경력 추출
      temp_club_career = driver.find_element_by_xpath('//*[@id="middle"]/div/div/div[4]/div[1]/div[2]/div[2]').text
      temp_club_career_2 = year_and_team_merge(temp_club_career)
      
      
      # 세부 설명 나오기
      driver.close()
      time.sleep(1)

      
      # 창 변경
      driver.switch_to.window(driver.window_handles[0])
      time.sleep(1)



      # 검색된 선수 숫자 세기
      player_list = driver.find_elements_by_xpath('//*[@id="divPlayerList"]')
      player_list_num = player_list[0].text.count(f'{search_name}')



      # 시즌, 이름, 급여, 오버롤, 속력, 가속력, 몸싸움, 스태미너 추출
      for num in range(1,player_list_num+1):
         # 이름 저장
         temp_name = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[2]').text
         name.append(temp_name)
         
         # 클럽 경력 저장
         # 같이 검색된 동명이인 처리
         if search_name == temp_name:
            club_career.append(temp_club_career_2)
         else:
            club_career.append('동명이인'+search_name)

         # 시즌 저장
         temp_season = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[1]/img')
         temp_season = temp_season.get_attribute('src').split('/')[-1].replace('.png','')
         season.append(temp_season)
   
         # 급여 저장
         temp_pay_side = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[2]/span').text
         pay_side.append(temp_pay_side)
       
         # 오버롤 저장
         temp_overall = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[3]/span/span').text
         overall.append(temp_overall)
         
         # 속력 저장
         temp_Sprint_speed = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[4]/span/span').text
         Sprint_speed.append(temp_Sprint_speed)
         
         # 가속력 저장
         temp_Acceleration = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[5]/span/span').text
         Acceleration.append(temp_Acceleration)
         
         # 몸싸움 저장
         temp_Strength = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[6]/span/span').text
         Strength.append(temp_Strength)
         
         # 스태미너 저장
         temp_Stamina = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[7]/span/span').text
         Stamina.append(temp_Stamina)
      

      # 검색어 초기화 
      driver.find_element_by_class_name('btn_reset').click()
      time.sleep(A)
   
   except:
      if driver.find_element_by_xpath('//*[@id="divPlayerList"]/div/span').text == '검색 결과가 없습니다.':
         error_name.append(search_name)
         print(f'error: {search_name}')
         
      else:
         break
      
             
# dict로 변형 & 데이터 프레임화
dict_temp = {  'name': [val for val in name],
               'pay_side': [val for val in pay_side],
               'overall': [val for val in overall],
               'Sprint_speed': [val for val in Sprint_speed],
               'Acceleration': [val for val in Acceleration],
               'Strength': [val for val in Strength],
               'Stamina': [val for val in Stamina],
               'season': [val for val in season],
               'club_career': [val for val in club_career]                            
}
columns = ['name','pay_side','overall','Sprint_speed','Acceleration','Strength','Stamina','season','club_career']

player_info_plus = pd.DataFrame(data = dict_temp, columns=columns)

  0%|          | 0/4233 [00:00<?, ?it/s]

In [118]:
# 데이터 확인
player_info_plus

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,W. 트로스트-에콩,12,81,76,65,89,65,COC,"[2018 ~ 2020 / 왓퍼드, 2017 ~ 2018 / 우디네세, 2015 ~..."
1,W. 트로스트-에콩,9,77,74,67,88,70,19PLS,"[2018 ~ 2020 / 왓퍼드, 2017 ~ 2018 / 우디네세, 2015 ~..."
2,W. 트로스트-에콩,5,67\n(65),63,53,79,63,LIVE,"[2018 ~ 2020 / 왓퍼드, 2017 ~ 2018 / 우디네세, 2015 ~..."
3,요나스 바게르,5,64,66,68,71,67,LIVE,[]
4,요나스 바게르,5,61,66,64,64,67,18,[]
...,...,...,...,...,...,...,...,...,...
9112,E. 마스트롤리아,5,58,43,45,52,41,18,[]
9113,F. 프로이아,5,60,71,64,70,65,LIVE,[]
9114,F. 프로이아,5,60,67,64,68,65,19,[]
9115,E. 스파고,5,52,63,54,64,62,21,[]


In [119]:
# 이전 csv 파일 불러오기
player_info_plus_before = pd.read_csv('./data/player_info/player_info_plus.csv')


# 데이터 프레임 병합
# 이전 csv 파일 불러오기
New_player_info_plus = pd.concat([player_info_plus_before,player_info_plus], ignore_index=True)
New_player_info_plus

# csv 파일로 저장
New_player_info_plus.to_csv("player_info_plus.csv", encoding='utf-8-sig', index = False)

In [120]:
# merge 전 파일 확인
player_info_plus_before

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,[]
1,조 에드워즈,5,60,62,64,65,83,LIVE,[]
2,J. 피에라치,5,60,59,55,68,66,18,[]
3,김도훈,21,99,96,89,106,90,TKL,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
4,김도훈,17,88,84,77,97,86,TKI,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
...,...,...,...,...,...,...,...,...,...
47326,S. 잉가손,13,82,73,74,81,81,EBS,"['2017 ~ 2019 / 파오크 FC', '2017 ~ 2017 / FC 로스토..."
47327,S. 잉가손,7,70,57,58,72,69,LIVE,"['2017 ~ 2019 / 파오크 FC', '2017 ~ 2017 / FC 로스토..."
47328,미르코 살비,5,62,34,42,62,36,18,[]
47329,미르코 살비,5,58,34,42,59,36,LIVE,[]


In [121]:
# merge 후 파일 확인
New_player_info_plus

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,[]
1,조 에드워즈,5,60,62,64,65,83,LIVE,[]
2,J. 피에라치,5,60,59,55,68,66,18,[]
3,김도훈,21,99,96,89,106,90,TKL,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
4,김도훈,17,88,84,77,97,86,TKI,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
...,...,...,...,...,...,...,...,...,...
56443,E. 마스트롤리아,5,58,43,45,52,41,18,[]
56444,F. 프로이아,5,60,71,64,70,65,LIVE,[]
56445,F. 프로이아,5,60,67,64,68,65,19,[]
56446,E. 스파고,5,52,63,54,64,62,21,[]


# 3. 데이터 후처리

In [130]:
# 데이터 불러오기
data = pd.read_csv('./data/player_info/player_info_plus.csv')
data

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,[]
1,조 에드워즈,5,60,62,64,65,83,LIVE,[]
2,J. 피에라치,5,60,59,55,68,66,18,[]
3,김도훈,21,99,96,89,106,90,TKL,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
4,김도훈,17,88,84,77,97,86,TKI,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
...,...,...,...,...,...,...,...,...,...
56443,E. 마스트롤리아,5,58,43,45,52,41,18,[]
56444,F. 프로이아,5,60,71,64,70,65,LIVE,[]
56445,F. 프로이아,5,60,67,64,68,65,19,[]
56446,E. 스파고,5,52,63,54,64,62,21,[]


In [131]:
# 결측값 확인
data.isna().sum()

name            0
pay_side        0
overall         0
Sprint_speed    0
Acceleration    0
Strength        0
Stamina         0
season          0
club_career     0
dtype: int64

## 3-1. 클럽 커리어 정리

In [ ]:
# 데이터 확인
data['club_career'].value_counts().to_frame()

In [ ]:
# 동명이인 검색 & 검토 필요
# ex) 로드리고 // '로드리고 벤탕쿠르'가 더 스텟높아서 먼저 나옴

In [ ]:
same_name_group = data['club_career'].str.contains('동명이인')
same_name_group

## 3-2. 라이브 부스트 효과 제거

In [132]:
# 데이터 확인
data['overall'].value_counts().to_frame()

,overall
59,2658
58,2550
57,2376
60,2373
56,2226
...,...
50\n(47),1
97\n(90),1
82\n(75),1
89\n(86),1


In [133]:
# 라이브 부스트 효과 받는 row 추출
live_boost_data = data[(data['overall'].str.contains('\n'))]
live_boost_data 

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
17,키어런 그린,5,56\n(54),59,61,63,77,LIVE,[]
19,박주원,5,60\n(59),44,43,60,33,LIVE,"['2013 ~ 2022 / 충남 아산 축구단', '2017 ~ 2018 / 대전 ..."
30,에반 은디카,16,90\n(89),93,84,91,86,22PLA,['2017 ~ 2018 / 아인트라흐트 프랑크푸르트']
32,에반 은디카,7,75\n(74),78,69,76,71,LIVE,['2017 ~ 2018 / 아인트라흐트 프랑크푸르트']
34,K. 애덤스 누후,5,65\n(64),61,59,84,62,LIVE,[]
...,...,...,...,...,...,...,...,...,...
56337,조나탕 이코네,7,73\n(72),83,84,52,68,LIVE,동명이인조나
56339,조나탕 클로스,6,72\n(71),77,79,60,82,LIVE,동명이인조나
56341,조나탕 그라디,5,71\n(69),69,70,74,72,LIVE,동명이인조나
56363,조나탄 오키타,5,64\n(63),79,77,67,69,LIVE,동명이인조나


In [134]:
# 라이브 부스트 받기 전/후 차이 구하기
def find_gap(num):
    LBD = live_boost_data['overall'].iloc[num].replace(')','').replace('(','').replace('\n','-')
    LBD_split = LBD.split('-')
    original_overall = LBD_split[1]
    gap = int(LBD_split[0]) - int(LBD_split[1])
    
    return gap, original_overall

# 라이브 부스트 받기 전 값으로 입력해주기
def remove_live_boost(original_overall,gap,num):
    live_boost_data['pay_side'].iloc[num] = str(live_boost_data['pay_side'].iloc[num])
    live_boost_data['overall'].iloc[num] = str(original_overall)
    live_boost_data['Sprint_speed'].iloc[num] = str(int(live_boost_data['Sprint_speed'].iloc[num]) - gap)
    live_boost_data['Acceleration'].iloc[num] = str(int(live_boost_data['Acceleration'].iloc[num]) - gap)
    live_boost_data['Strength'].iloc[num] = str(int(live_boost_data['Strength'].iloc[num]) - gap)
    live_boost_data['Stamina'].iloc[num] = str(int(live_boost_data['Stamina'].iloc[num]) - gap)

In [135]:
# 라이브 부스트 효과 제거 및 data 
# 업데이트
for num in range(len(live_boost_data)):
    gap, original_overall = find_gap(num)
    
    remove_live_boost(original_overall,gap,num)

data.update(live_boost_data)
data

  0%|          | 0/4323 [00:00<?, ?it/s]

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,[]
1,조 에드워즈,5,60,62,64,65,83,LIVE,[]
2,J. 피에라치,5,60,59,55,68,66,18,[]
3,김도훈,21,99,96,89,106,90,TKL,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
4,김도훈,17,88,84,77,97,86,TKI,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
...,...,...,...,...,...,...,...,...,...
56443,E. 마스트롤리아,5,58,43,45,52,41,18,[]
56444,F. 프로이아,5,60,71,64,70,65,LIVE,[]
56445,F. 프로이아,5,60,67,64,68,65,19,[]
56446,E. 스파고,5,52,63,54,64,62,21,[]
